# Manipulating Cards

In [7]:
#cards are represented as integers from 0-52 with 0 being an Ace of suit 0 (the actual suits do not matter in Cribbage, however 
# some points do rely on suits matching) and 52 being a King of suit 3

# given a card (integer), return the rank with 1 denoting Ace, 11 denoting Jack, 12 denoting Queen, 13 denoting King, and each other number denoting itself
def rank(n):
    return ((n)//4)+1

# given a card (integer), return the suit from 0-3
def suit(n):
    return (n)%4

# given a card (integer), return the value with Ace valued at 1, all face cards valued at 10, and every other card the same value as its rank
def value(n):
    if ((n)//4)+1 < 10:
        return ((n)//4)+1
    else:
        return 10


# Scoring a Hand

In [8]:
#scoring rules from Wikipedia


# flush: four points for a flush, where all four cards in the hand are of the same suit, with an additional point if the starter card is also of 
# that suit.
def scoreFlush(handArr):
    for card in handArr[2:]:
        if suit(card) != suit(handArr[1]):
            return 0
    if suit(handArr[0]) == suit(handArr[1]):
        return 5
    else:
        return 4

# nob: one point for holding the Jack of the same suit as the starter card

def nob(handArr):
    for card in handArr[1:]:
        if rank(card) == 11:
            if suit(card) == suit(handArr[0]):
                return True
    else:
        return False

# fifteen: two points for each separate combination of two or more cards totalling exactly fifteen
def scoreFifteens(handArr, quads, triples, pairs):
    score = 0
    if value(handArr[0]) + value(handArr[1]) + value(handArr[2]) + value(handArr[3]) + value(handArr[4]) == 15:
        score += 2

  
    for a,b,c,d in quads:
        if value(a) + value(b) + value(c) + value(d) == 15:
            score += 2

    for a,b,c in triples:
        if value(a) + value(b) + value(c) == 15:
            score += 2
    
    for a,b in pairs:
        if value(a) + value(b) == 15:
            score += 2
            
    return score

# pairs: two points for a pair of cards of a kind, six points for three cards of a kind (known as a "pair royal", comprising three distinct pairs), 
# twelve points for four cards of a kind (a "double pair royal", comprising six distinct pairs)
def scorePairs(pairs):
    score = 0
    for a,b in pairs:
        if rank(a) == rank(b):
            score += 2
    return score

# runs: three points for a run of three consecutive cards (regardless of suit), four points for a run of four, five points for a run of five

def scoreRuns(handArr, quads, triples):
    score = 0

    handArr.sort()
    if rank(handArr[0]) + 1 == rank(handArr[1]) and rank(handArr[1]) + 1 == rank(handArr[2]) and rank(handArr[2]) + 1 == rank(handArr[3]) and rank(handArr[3]) + 1 == rank(handArr[4]):
        return 5
   
    for a,b,c,d in quads:
        if rank(a) + 1 == rank(b) and rank(b) + 1 == rank(c) and rank(c) + 1 == rank(d):
            score += 4
    
    if score > 0:
        return score
    
    for a,b,c in triples:
        if rank(a) + 1 == rank(b) and rank(b) + 1 == rank(c):
            score += 3
    return score



In [9]:

def scoreHand(handArr):
    handArr = list(handArr)
    score = 0

    # scoring flush
    score += scoreFlush(handArr)
    
    # scoring nob
    if nob(handArr):
        score += 1
   

    #make pairs
    pairs = []
    alreadyused = []
    for i in handArr:
        for j in handArr:
            if i != j and j not in alreadyused:
                pairs.append((i,j))
        alreadyused.append(i)
   
    #make triples
    triples = []
    alreadyusedk = []
    for k in handArr:
        for i,j in pairs:
            if k not in [i,j] and i not in alreadyusedk and j not in alreadyusedk:
                triple = tuple(sorted([k,i,j]))
                triples.append(triple)
        alreadyusedk.append(k)
  
    #make quads
    quads = []
    alreadyusedm = []
    for m in handArr:
        for i,j,k in triples:
            if m not in [i,j,k] and i not in alreadyusedm and j not in alreadyusedm and k not in alreadyusedm:
                quad = tuple(sorted([k,i,j,m]))
                quads.append(quad)
        alreadyusedm.append(m)

    #score pairs
    score += scorePairs(pairs)

    #score runs
    score += scoreRuns(handArr, quads, triples)
             
    #score fifteens
    score += scoreFifteens(handArr, quads, triples, pairs)
    
    if score == 29:
        print("This hand scores 29: " + str(list(map(lambda x:rank(x),handArr))))
    
    return score

# Finding All Possible Scores in Cribbage

In [10]:
# generate every possible hand in Cribbage, included the turned up card, and calculate all possible scores
def generateCribbageScores():
    scores = {}
    cards = list(range(52))
    

    #make pairs
    pairs = {}
    alreadyused = []
    for i in cards:
        for j in cards:
            if i != j and j not in alreadyused:
                pair = tuple(sorted([i,j]))
                pairs[pair] = "0"
        alreadyused.append(i)
   
    #make triples
    triples = {}
    alreadyusedk = []
    for k in cards:
        for i,j in pairs:
            if k not in [i,j] and i not in alreadyusedk and j not in alreadyusedk:
                triple = tuple(sorted([k,i,j]))
                triples[triple] = "0"
        alreadyusedk.append(k)
  
    #make quads
    quads = {}
    alreadyusedm = []
    for m in cards:
        for i,j,k in triples:
            if m not in [i,j,k] and i not in alreadyusedm and j not in alreadyusedm and k not in alreadyusedm:
                quad = tuple(sorted([k,i,j,m]))
                quads[quad] = "0"
        alreadyusedm.append(m)

    #make hands
    # the first card is the turned up card so the order matters
    hands = {}
    for n in cards:
        for a,b,c,d in quads:
            if n not in [a,b,c,d]:
                hands[n,a,b,c,d] = "0"
    
    # score each hand
    for hand in hands:
        currScore = scoreHand(hand)
        if currScore in scores:
            scores[currScore] += 1
        else:
            scores[currScore] = 1

    return scores

# Results

In [11]:
scores = generateCribbageScores()

This hand scores 29: [5, 5, 5, 5, 11]
This hand scores 29: [5, 5, 5, 5, 11]
This hand scores 29: [5, 5, 5, 5, 11]
This hand scores 29: [5, 5, 5, 5, 11]


In [12]:
print(scores)
print(sorted(scores))

{12: 317340, 13: 19656, 8: 1137236, 15: 9168, 6: 1800268, 9: 361224, 10: 388740, 7: 751324, 14: 90100, 16: 58248, 4: 2855676, 2: 2813796, 5: 697508, 3: 505008, 11: 51680, 17: 11196, 20: 8068, 0: 1009008, 1: 99792, 24: 3680, 18: 2708, 21: 2496, 23: 356, 28: 76, 29: 4, 22: 444}
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 28, 29]
